
# Sistema de Planificación de Viajes

Alumna: Yvonne Echevarria.


In [1]:
# CELDA 1: Instalación de Dependencias para OpenAI
!pip install langgraph langchain langchain-openai wikipedia termcolor python-dotenv -q

print("✅ Dependencias instaladas correctamente")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.5 MB/s eta 0:00:00
✅ Dependencias instaladas correctamente


In [4]:

    import os
    from termcolor import colored
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    print(colored("✅ API Key cargada desde secretos de Colab", "green"))

🔑 CONFIGURANDO API KEYS...

✅ OpenAI API Key cargada desde secretos
✅ Google API Key cargada desde secretos

✅ SISTEMA LISTO - Usaremos OpenAI


In [13]:
# CELDA 3: Configuración de LLM y Herramientas (OPENAI)
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage
import wikipedia

print(colored("🤖 CONFIGURANDO OPENAI...\n", "cyan", attrs=["bold"]))

# ============================================================
# CONFIGURAR OPENAI
# ============================================================
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Modelo económico
    api_key=OPENAI_API_KEY,
    temperature=0.7
)

print(colored("✅ OpenAI (GPT-4o-mini) configurado", "green"))

# ============================================================
# FUNCIÓN DE BÚSQUEDA EN WIKIPEDIA
# ============================================================
def buscar_destino_wikipedia(destino):
    """
    Busca información sobre un destino turístico en Wikipedia.
    """
    try:
        wikipedia.set_lang("es")
        resumen = wikipedia.summary(destino, sentences=10)
        return f"📍 Información sobre {destino}:\n\n{resumen}"

    except wikipedia.exceptions.DisambiguationError as e:
        print(colored(f"⚠️  Múltiples resultados para '{destino}', usando: {e.options[0]}", "yellow"))
        resumen = wikipedia.summary(e.options[0], sentences=10)
        return f"📍 Información sobre {e.options[0]}:\n\n{resumen}"

    except wikipedia.exceptions.PageError:
        return f"❌ No se encontró información sobre '{destino}' en Wikipedia."

    except Exception as e:
        return f"❌ Error al buscar información: {str(e)}"

print(colored("✅ Función de Wikipedia lista", "green"))

# ============================================================
# PRUEBA
# ============================================================
print(colored("\n🧪 Prueba rápida:", "cyan"))
try:
    test_response = llm.invoke([HumanMessage(content="Di solo: Sistema OK")])
    print(colored(f"   Respuesta: {test_response.content}", "green"))
    print(colored("\n✅ TODO LISTO PARA USAR CON OPENAI", "green", attrs=["bold"]))
except Exception as e:
    print(colored(f"   ❌ Error: {str(e)}", "red"))


🤖 CONFIGURANDO OPENAI...

✅ OpenAI (GPT-4o-mini) configurado
✅ Función de Wikipedia lista

🧪 Prueba rápida:
   Respuesta: Sistema OK

✅ TODO LISTO PARA USAR CON OPENAI


In [14]:
# CELDA 4: Definición del Estado Compartido
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class TravelState(TypedDict):
    """
    Estado compartido entre todos los agentes del sistema.

    Este estado viaja por todo el grafo y cada agente puede:
    - Leer información
    - Agregar/modificar información
    - Acumular mensajes conversacionales
    """

    # ============ CONVERSACIÓN ============
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # Historial completo de la conversación usuario-sistema
    # add_messages es una función de REDUCCIÓN que concatena mensajes

    # ============ INFORMACIÓN DEL VIAJE ============
    destination: str  # Ciudad/país de destino (ej: "París")
    duration: str     # Duración del viaje (ej: "5 días")
    activities: str   # Tipo de actividades (ej: "museos y gastronomía")
    budget: str       # Presupuesto aproximado (ej: "medio")

    # ============ DECISIONES DE FLUJO ============
    supervisor_decision: str  # "CONTINUE" o "INVESTIGATE"
    interview_complete: bool  # ¿Tenemos toda la info del usuario?

    # ============ RESULTADOS ============
    destination_research: str  # Info de Wikipedia sobre el destino
    travel_itinerary: str      # Itinerario final día por día


print(colored("✅ Estado compartido definido", "green"))
print(colored("📊 Variables del estado:", "cyan"))
print(colored("   - messages: Historial conversacional", "white"))
print(colored("   - destination: Destino del viaje", "white"))
print(colored("   - duration: Duración del viaje", "white"))
print(colored("   - activities: Tipo de actividades", "white"))
print(colored("   - budget: Presupuesto", "white"))
print(colored("   - supervisor_decision: Control de flujo", "white"))
print(colored("   - interview_complete: Flag de completitud", "white"))
print(colored("   - destination_research: Info del destino", "white"))
print(colored("   - travel_itinerary: Itinerario final", "white"))

✅ Estado compartido definido
📊 Variables del estado:
   - messages: Historial conversacional
   - destination: Destino del viaje
   - duration: Duración del viaje
   - activities: Tipo de actividades
   - budget: Presupuesto
   - supervisor_decision: Control de flujo
   - interview_complete: Flag de completitud
   - destination_research: Info del destino
   - travel_itinerary: Itinerario final


In [15]:
# CELDA 5: Prompts Especializados
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ============================================================
# PROMPT 1: ENTREVISTADOR DE VIAJES
# ============================================================
interviewer_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Eres un agente de viajes experto, amigable y entusiasta.

TU MISIÓN:
1. Descubrir QUÉ quiere el viajero (destino, actividades, estilo de viaje)
2. Entender CUÁNTO tiempo tiene (duración del viaje)
3. Conocer sus PREFERENCIAS (tipo de actividades)
4. Evaluar su PRESUPUESTO aproximado

CÓMO DEBES ACTUAR:
- Haz UNA pregunta a la vez (no abrumes)
- Sé conversacional y natural
- Muestra entusiasmo por los destinos
- Da opciones cuando sea útil

INFORMACIÓN QUE NECESITAS RECOPILAR:
✈️ Destino: ¿A dónde quiere viajar?
📅 Duración: ¿Cuántos días?
🎯 Actividades: ¿Qué le gusta? (cultura, aventura, playa, gastronomía, etc.)
💰 Presupuesto: ¿Aproximado? (bajo, medio, alto)

IMPORTANTE: Cuando tengas TODA esta información clara, termina tu mensaje diciendo EXACTAMENTE:
"ENTREVISTA_COMPLETA: Tengo todo lo necesario para planificar tu viaje."

Si falta alguna información, sigue preguntando de forma natural.
"""),
    MessagesPlaceholder(variable_name="messages")
])

# ============================================================
# PROMPT 2: SUPERVISOR
# ============================================================
supervisor_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Eres un supervisor que detecta cuando el entrevistador tiene toda la información necesaria.

Analiza SOLO el último mensaje del entrevistador.

REGLAS SIMPLES:
- Si el mensaje contiene "ENTREVISTA_COMPLETA:" → responde exactamente: "INVESTIGATE"
- Si no contiene esa frase → responde exactamente: "CONTINUE"

Responde SOLO con una palabra: "INVESTIGATE" o "CONTINUE"
Sin puntos, sin explicaciones, solo la palabra.
"""),
    MessagesPlaceholder(variable_name="messages")
])

# ============================================================
# PROMPT 3: INVESTIGADOR DE DESTINOS
# ============================================================
wikipedia_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Eres un investigador de destinos turísticos experto.

Recibirás información de Wikipedia sobre un destino.

TU TRABAJO:
1. Extraer los datos MÁS RELEVANTES para un viajero:
   - Qué ver (atracciones principales)
   - Cultura y costumbres importantes
   - Clima típico
   - Tips de viaje

2. Organizar la información de forma clara y útil

3. Enfocarte en lo PRÁCTICO, no en historia extensa

Devuelve un resumen estructurado y fácil de leer.
"""),
    MessagesPlaceholder(variable_name="messages")
])

# ============================================================
# PROMPT 4: GENERADOR DE ITINERARIO
# ============================================================
study_guide_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Eres un planificador de viajes profesional y creativo.

Vas a crear un ITINERARIO DÍA POR DÍA personalizado.

INFORMACIÓN QUE TENDRÁS:
- Destino del viaje
- Duración (número de días)
- Preferencias de actividades
- Presupuesto
- Información del destino (de Wikipedia)

CÓMO CREAR EL ITINERARIO:

1. ESTRUCTURA:
   DÍA 1: [Título del día]
   - Mañana: [Actividades]
   - Tarde: [Actividades]
   - Noche: [Actividades]

   DÍA 2: [Título del día]
   ...

2. PERSONALIZACIÓN:
   - Adapta las actividades a las preferencias del usuario
   - Respeta el presupuesto (bajo = actividades gratuitas, medio = mix, alto = premium)
   - Incluye variedad: no solo museos, no solo restaurantes

3. DETALLES ÚTILES:
   - Nombres de lugares específicos
   - Tips prácticos (mejor hora para visitar, cómo llegar)
   - Alternativas por si llueve

4. FORMATO:
   - Usa emojis para hacerlo visual
   - Sé específico pero conciso
   - Incluye estimado de tiempo

Crea un itinerario que el viajero pueda seguir fácilmente.
"""),
    MessagesPlaceholder(variable_name="messages")
])

print(colored("✅ Prompts especializados definidos", "green"))
print(colored("🎭 4 Agentes configurados:", "cyan"))
print(colored("   1. 👤 Entrevistador de Viajes", "white"))
print(colored("   2. 🔍 Supervisor de Completitud", "white"))
print(colored("   3. 🗺️  Investigador de Destinos", "white"))
print(colored("   4. 📋 Generador de Itinerario", "white"))

✅ Prompts especializados definidos
🎭 4 Agentes configurados:
   1. 👤 Entrevistador de Viajes
   2. 🔍 Supervisor de Completitud
   3. 🗺️  Investigador de Destinos
   4. 📋 Generador de Itinerario


In [30]:
# CELDA 6: Implementación de Nodos
from langchain_core.messages import HumanMessage, AIMessage

# ============================================================
# NODO 1: ENTREVISTADOR DE VIAJES
# ============================================================
async def interviewer_node(state: TravelState):
    """
    Agente que entrevista al usuario sobre sus preferencias de viaje.
    """
    print(colored("\n👤 [ENTREVISTADOR ACTIVADO]", "cyan"))

    messages = state["messages"]

    # Primera interacción: saludo inicial
    if not messages or len(messages) == 0:
        initial_greeting = AIMessage(content="""¡Hola! 👋 Soy tu asistente personal de viajes.

Estoy aquí para ayudarte a planificar el viaje perfecto. Para hacerlo, necesito conocer un poco sobre tus preferencias.

¿A qué destino te gustaría viajar? 🌍✈️""")
        return {"messages": [initial_greeting]}

    # Turnos siguientes: usar el LLM
    response = await (interviewer_prompt | llm).ainvoke({"messages": messages})
    return {"messages": [response]}


# ============================================================
# NODO 2: SUPERVISOR (VERSIÓN MEJORADA - MÁS FLEXIBLE)
# ============================================================
async def supervisor_node(state: TravelState):
    """
    Supervisa si la entrevista está completa.

    Versión mejorada que detecta múltiples señales de finalización.
    """
    print(colored("\n🔍 [SUPERVISOR ACTIVADO]", "yellow"))

    messages = state["messages"]

    # Obtener el último mensaje del entrevistador
    last_message = ""
    for msg in reversed(messages):
        if hasattr(msg, 'content') and msg.content:
            last_message = msg.content.lower()  # Convertir a minúsculas
            break

    # Detectar múltiples señales de finalización
    completion_signals = [
        "entrevista_completa",
        "entrevista completa",
        "tengo todo lo necesario",
        "toda la información",
        "información completa",
        "listo para planificar",
        "proceder con la investigación",
        "procederemos con",
    ]

    # Verificar si alguna señal está presente
    is_complete = any(signal in last_message for signal in completion_signals)

    # TAMBIÉN: Verificar si ya tenemos las 4 preguntas respondidas
    # (como respaldo si el LLM no dice la frase exacta)
    conversation_text = "\n".join([
        msg.content for msg in messages
        if hasattr(msg, 'content')
    ]).lower()

    has_destination = any(word in conversation_text for word in ["destino", "viajar a", "ir a", "visitar"])
    has_duration = any(word in conversation_text for word in ["días", "semanas", "meses", "tiempo"])
    has_activities = any(word in conversation_text for word in ["actividades", "hacer", "visitar", "ver", "cultura", "aventura", "playa"])
    has_budget = any(word in conversation_text for word in ["presupuesto", "bajo", "medio", "alto", "dinero"])

    # Contar cuántas preguntas se han hecho
    assistant_messages = [msg for msg in messages if hasattr(msg, 'type') and msg.type == 'ai']
    questions_asked = len(assistant_messages)

    # Si tiene señal de completitud O (tiene las 4 respuestas Y ha hecho al menos 4 preguntas)
    if is_complete or (has_destination and has_duration and has_activities and has_budget and questions_asked >= 4):
        decision = "INVESTIGATE"
        print(colored("   ✅ Entrevista completa detectada", "green"))
        if not is_complete:
            print(colored("   ℹ️  Detectado por respaldo (4 respuestas completas)", "cyan"))
    else:
        decision = "CONTINUE"
        print(colored("   ⏳ Continuando entrevista...", "yellow"))
        print(colored(f"   📊 Preguntas respondidas: Destino:{has_destination} Duración:{has_duration} Actividades:{has_activities} Presupuesto:{has_budget}", "cyan"))

    return {
        "supervisor_decision": decision,
        "interview_complete": (decision == "INVESTIGATE")
    }


# ============================================================
# NODO 3: INVESTIGADOR DE DESTINOS (VERSIÓN MEJORADA)
# ============================================================
async def wikipedia_node(state: TravelState):
    """
    Busca información sobre el destino en Wikipedia.
    Versión mejorada con mejor extracción de destino.
    """
    print(colored("\n🗺️  [INVESTIGADOR DE DESTINOS ACTIVADO]", "magenta"))

    messages = state["messages"]

    # PASO 1: Extraer TODA la conversación
    full_conversation = "\n".join([
        f"{msg.type}: {msg.content}"
        for msg in messages
        if hasattr(msg, 'content') and hasattr(msg, 'type')
    ])

    print(colored(f"   📝 Conversación completa capturada ({len(messages)} mensajes)", "cyan"))

    # PASO 2: Usar el LLM para extraer TODA la información
    extraction_prompt = f"""
Analiza esta conversación sobre un viaje y extrae la siguiente información:

CONVERSACIÓN:
{full_conversation}

EXTRAE:
1. Destino (ciudad o país):
2. Duración del viaje:
3. Tipo de actividades:
4. Presupuesto:

Responde en formato:
DESTINO: [ciudad/país]
DURACIÓN: [X días]
ACTIVIDADES: [lista]
PRESUPUESTO: [bajo/medio/alto]
"""

    extraction_response = await llm.ainvoke([HumanMessage(content=extraction_prompt)])
    extracted_info = extraction_response.content

    print(colored(f"   📊 Información extraída:", "cyan"))
    print(colored(f"   {extracted_info}", "white"))

    # Extraer solo el destino para Wikipedia
    destino_lines = [line for line in extracted_info.split('\n') if 'DESTINO:' in line.upper()]
    if destino_lines:
        destino = destino_lines[0].split(':', 1)[1].strip()
    else:
        # Fallback: buscar en los mensajes del usuario
        user_messages = [msg.content for msg in messages if hasattr(msg, 'type') and msg.type == 'human']
        destino = user_messages[0] if user_messages else "destino desconocido"

    print(colored(f"   📍 Destino identificado: {destino}", "cyan"))

    # PASO 3: Buscar en Wikipedia
    print(colored(f"   🔎 Buscando información en Wikipedia...", "cyan"))
    wikipedia_info = buscar_destino_wikipedia(destino)

    # PASO 4: Procesar información con el LLM
    print(colored(f"   ⚙️  Procesando información...", "cyan"))
    research_prompt = f"""
Información de Wikipedia sobre {destino}:

{wikipedia_info}

Tu trabajo: Extrae y organiza la información MÁS RELEVANTE para un viajero:
- Qué ver (atracciones principales)
- Cultura y costumbres
- Clima típico
- Tips de viaje

Sé específico y práctico.
"""

    processed = await (wikipedia_prompt | llm).ainvoke([
        HumanMessage(content=research_prompt)
    ])

    print(colored(f"   ✅ Investigación completada", "green"))

    # Guardar TODA la información extraída en el estado
    return {
        "destination": destino,
        "destination_research": processed.content,
        "duration": extracted_info,  # Guardamos toda la info extraída
    }


# ============================================================
# NODO 4: GENERADOR DE ITINERARIO (VERSIÓN MEJORADA)
# ============================================================
async def study_guide_node(state: TravelState):
    """
    Genera el itinerario completo día por día.
    Versión mejorada con mejor captura de información.
    """
    print(colored("\n📋 [GENERADOR DE ITINERARIO ACTIVADO]", "green"))

    # Extraer información del estado
    messages = state["messages"]
    destino = state.get("destination", "destino desconocido")
    research = state.get("destination_research", "Sin información disponible")

    # Construir TODA la conversación
    full_conversation = "\n".join([
        f"{msg.type}: {msg.content}"
        for msg in messages
        if hasattr(msg, 'content') and hasattr(msg, 'type')
    ])

    print(colored(f"   📊 Destino: {destino}", "cyan"))
    print(colored(f"   📝 Conversación: {len(messages)} mensajes", "cyan"))

    # Prompt mejorado para generar el itinerario
    itinerary_prompt = f"""
Crea un itinerario de viaje PERSONALIZADO Y ESPECÍFICO.

CONVERSACIÓN CON EL USUARIO:
{full_conversation}

INFORMACIÓN DEL DESTINO:
{research}

INSTRUCCIONES IMPORTANTES:
1. LEE CUIDADOSAMENTE la conversación para identificar:
   - Destino exacto
   - Duración del viaje (número de días)
   - Tipo de actividades que le interesan
   - Presupuesto (bajo/medio/alto)

2. Crea un itinerario DÍA POR DÍA con:
   - Mañana: actividades específicas
   - Tarde: actividades específicas
   - Noche: actividades específicas

3. Personaliza según:
   - Preferencias mencionadas en la conversación
   - Presupuesto indicado
   - Tipo de viajero

4. Usa emojis para hacerlo visual

5. Incluye nombres REALES de lugares, no genéricos

GENERA EL ITINERARIO COMPLETO AHORA:
"""

    print(colored(f"   ⚙️  Generando itinerario para {destino}...", "cyan"))

    # Generar itinerario con el LLM
    itinerary_response = await llm.ainvoke([HumanMessage(content=itinerary_prompt)])

    print(colored(f"   ✅ Itinerario generado exitosamente", "green"))

    return {"travel_itinerary": itinerary_response.content}


print(colored("✅ Todos los nodos implementados correctamente", "green"))
print(colored("🔧 4 Nodos listos:", "cyan"))
print(colored("   1. interviewer_node - Entrevista al usuario", "white"))
print(colored("   2. supervisor_node - Detecta completitud", "white"))
print(colored("   3. wikipedia_node - Investiga destino", "white"))
print(colored("   4. study_guide_node - Genera itinerario", "white"))

✅ Todos los nodos implementados correctamente
🔧 4 Nodos listos:
   1. interviewer_node - Entrevista al usuario
   2. supervisor_node - Detecta completitud
   3. wikipedia_node - Investiga destino
   4. study_guide_node - Genera itinerario


In [17]:
# CELDA 7: Función de Routing
from typing import Literal

def supervisor_router(state: TravelState) -> Literal["CONTINUE", "INVESTIGATE"]:
    """
    Router que decide el siguiente paso basado en la decisión del supervisor.

    Input: Estado actual
    Output: String que indica el camino a seguir

    Opciones:
    - "CONTINUE": Volver al entrevistador (falta información)
    - "INVESTIGATE": Ir a buscar info del destino (entrevista completa)
    """

    # Obtener la decisión del supervisor
    decision = state.get("supervisor_decision", "CONTINUE")

    print(colored(f"\n🎯 [ROUTER ACTIVADO]", "blue"))
    print(colored(f"   Decisión del supervisor: {decision}", "blue"))

    # Routing condicional
    if decision == "INVESTIGATE":
        print(colored(f"   → Ruta: Ir a INVESTIGADOR 🗺️", "green"))
        return "INVESTIGATE"
    else:
        print(colored(f"   → Ruta: Volver a ENTREVISTADOR 👤", "yellow"))
        return "CONTINUE"


print(colored("✅ Función de routing implementada", "green"))
print(colored("🔀 Decisiones posibles:", "cyan"))
print(colored("   - CONTINUE → Volver al entrevistador", "white"))
print(colored("   - INVESTIGATE → Ir al investigador", "white"))

✅ Función de routing implementada
🔀 Decisiones posibles:
   - CONTINUE → Volver al entrevistador
   - INVESTIGATE → Ir al investigador


In [28]:
# CELDA 8: Construcción del Grafo (con visualización)
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

def create_travel_system():
    """
    Construye el sistema multi-agente completo.
    """

    print(colored("\n🏗️  [CONSTRUYENDO GRAFO]", "cyan", attrs=["bold"]))

    # ============================================================
    # PASO 1: Crear el grafo con el tipo de estado
    # ============================================================
    workflow = StateGraph(TravelState)
    print(colored("   ✓ Grafo inicializado con TravelState", "green"))

    # ============================================================
    # PASO 2: Agregar los 4 nodos (agentes)
    # ============================================================
    workflow.add_node("interviewer", interviewer_node)
    workflow.add_node("supervisor", supervisor_node)
    workflow.add_node("wikipedia", wikipedia_node)
    workflow.add_node("study_guide", study_guide_node)
    print(colored("   ✓ 4 nodos agregados al grafo", "green"))

    # ============================================================
    # PASO 3: Definir el punto de entrada
    # ============================================================
    workflow.add_edge(START, "interviewer")
    print(colored("   ✓ Punto de entrada: START → interviewer", "green"))

    # ============================================================
    # PASO 4: Flujo lineal simple (determinístico)
    # ============================================================
    workflow.add_edge("interviewer", "supervisor")
    print(colored("   ✓ Arista: interviewer → supervisor", "green"))

    # ============================================================
    # PASO 5: Decisión condicional del supervisor (CLAVE)
    # ============================================================
    workflow.add_conditional_edges(
        "supervisor",
        supervisor_router,
        {
            "CONTINUE": "interviewer",
            "INVESTIGATE": "wikipedia"
        }
    )
    print(colored("   ✓ Routing condicional configurado:", "green"))
    print(colored("     - CONTINUE → interviewer (loop)", "yellow"))
    print(colored("     - INVESTIGATE → wikipedia (avanzar)", "yellow"))

    # ============================================================
    # PASO 6: Flujo lineal después de Wikipedia
    # ============================================================
    workflow.add_edge("wikipedia", "study_guide")
    workflow.add_edge("study_guide", END)
    print(colored("   ✓ Aristas finales: wikipedia → study_guide → END", "green"))

    # ============================================================
    # PASO 7: Configurar memoria persistente (checkpointing)
    # ============================================================
    memory = MemorySaver()
    print(colored("   ✓ Memoria persistente configurada", "green"))

    # ============================================================
    # PASO 8: Compilar el grafo
    # ============================================================
    compiled_graph = workflow.compile(checkpointer=memory)
    print(colored("\n✅ GRAFO COMPILADO EXITOSAMENTE", "green", attrs=["bold"]))

    return compiled_graph


# Crear el sistema
travel_graph = create_travel_system()

print(colored("\n" + "="*60, "cyan"))
print(colored("🎉 SISTEMA MULTI-AGENTE LISTO PARA USAR", "cyan", attrs=["bold"]))
print(colored("="*60, "cyan"))

# ============================================================
# VISUALIZACIÓN ASCII DEL GRAFO (NUEVO)
# ============================================================
print(colored("\n📊 ARQUITECTURA DEL SISTEMA:", "yellow", attrs=["bold"]))
print(colored("""
        ┌─────────┐
        │  START  │
        └────┬────┘
             ↓
      ┌─────────────┐
      │ interviewer │ ←──────┐
      └──────┬──────┘        │
             ↓                │
      ┌─────────────┐         │
      │ supervisor  │         │
      └──────┬──────┘         │
             ↓                │
        [ROUTER]              │
             ↓                │
       ¿CONTINUE? ────────────┘
             │
       ¿INVESTIGATE?
             ↓
      ┌─────────────┐
      │  wikipedia  │
      └──────┬──────┘
             ↓
      ┌─────────────┐
      │ study_guide │
      └──────┬──────┘
             ↓
        ┌─────────┐
        │   END   │
        └─────────┘
""", "cyan"))

print(colored("🔄 Nodos: 4", "white"))
print(colored("   • interviewer (Entrevistador)", "white"))
print(colored("   • supervisor (Controlador)", "white"))
print(colored("   • wikipedia (Investigador)", "white"))
print(colored("   • study_guide (Generador)", "white"))

print(colored("\n🔀 Conexiones: 6", "white"))
print(colored("   • START → interviewer", "white"))
print(colored("   • interviewer → supervisor", "white"))
print(colored("   • supervisor → interviewer (si CONTINUE)", "white"))
print(colored("   • supervisor → wikipedia (si INVESTIGATE)", "white"))
print(colored("   • wikipedia → study_guide", "white"))
print(colored("   • study_guide → END", "white"))

print(colored("\n" + "="*60 + "\n", "cyan"))


🏗️  [CONSTRUYENDO GRAFO]
   ✓ Grafo inicializado con TravelState
   ✓ 4 nodos agregados al grafo
   ✓ Punto de entrada: START → interviewer
   ✓ Arista: interviewer → supervisor
   ✓ Routing condicional configurado:
     - CONTINUE → interviewer (loop)
     - INVESTIGATE → wikipedia (avanzar)
   ✓ Aristas finales: wikipedia → study_guide → END
   ✓ Memoria persistente configurada

✅ GRAFO COMPILADO EXITOSAMENTE

🎉 SISTEMA MULTI-AGENTE LISTO PARA USAR

📊 ARQUITECTURA DEL SISTEMA:

        ┌─────────┐
        │  START  │
        └────┬────┘
             ↓
      ┌─────────────┐
      │ interviewer │ ←──────┐
      └──────┬──────┘        │
             ↓                │
      ┌─────────────┐         │
      │ supervisor  │         │
      └──────┬──────┘         │
             ↓                │
        [ROUTER]              │
             ↓                │
       ¿CONTINUE? ────────────┘
             │
       ¿INVESTIGATE?
             ↓
      ┌─────────────┐
      │  wikipedia  │
    

In [36]:
# CELDA 9: Playground Interactivo (VERSIÓN CON STREAMING)
import uuid
from langchain_core.messages import HumanMessage

async def run_travel_planner():
    """
    Ejecuta el sistema de planificación de viajes de forma interactiva.
    """

    print(colored("""
╔══════════════════════════════════════════════════════════╗
║                                                          ║
║    ✈️  PLANIFICADOR DE VIAJES MULTI-AGENTE ✈️           ║
║                                                          ║
║    Sistema: LangGraph                                    ║
║    Agentes: 4 especializados                             ║
║    Modo: Interactivo                                     ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝
    """, "cyan", attrs=["bold"]))

    print(colored("💡 Tip: El sistema te hará preguntas para planificar tu viaje", "yellow"))
    print(colored("💡 Responde naturalmente, como si hablaras con un agente de viajes\n", "yellow"))

    # ============================================================
    # CONFIGURACIÓN DE SESIÓN
    # ============================================================
    thread_id = f"travel_session_{uuid.uuid4().hex[:8]}"
    config = {
        "configurable": {"thread_id": thread_id},
        "recursion_limit": 50
    }

    print(colored(f"🔑 Sesión iniciada: {thread_id}\n", "cyan"))

    # ============================================================
    # ESTADO INICIAL
    # ============================================================
    current_state = {
        "messages": [],
        "destination": "",
        "duration": "",
        "activities": "",
        "budget": "",
        "supervisor_decision": "CONTINUE",
        "interview_complete": False,
        "destination_research": "",
        "travel_itinerary": ""
    }

    # ============================================================
    # PRIMERA EJECUCIÓN: Obtener saludo inicial
    # ============================================================
    print(colored("🔄 Iniciando conversación...\n", "cyan"))

    # Ejecutar una vez para obtener el saludo
    async for event in travel_graph.astream(current_state, config):
        for node_name, node_output in event.items():
            if node_name == "interviewer":
                current_state = node_output
                break
        break  # Solo queremos el primer mensaje

    # ============================================================
    # LOOP PRINCIPAL DE CONVERSACIÓN
    # ============================================================
    while True:

        # Mostrar el último mensaje del asistente
        if current_state.get("messages"):
            last_message = current_state["messages"][-1]
            if hasattr(last_message, 'content') and hasattr(last_message, 'type'):
                if last_message.type == 'ai':
                    print(colored(f"🤖 Asistente: {last_message.content}\n", "cyan"))

        # Verificar si ya terminamos
        if current_state.get("interview_complete", False):
            print(colored("\n" + "="*60, "magenta"))
            print(colored("✅ PLANIFICACIÓN COMPLETADA", "green", attrs=["bold"]))
            print(colored("="*60 + "\n", "magenta"))

            # Continuar ejecutando hasta el final (wikipedia + study_guide)
            async for event in travel_graph.astream(current_state, config):
                for node_name, node_output in event.items():
                    current_state = node_output

            # Mostrar resultados finales
            if current_state.get("destination_research"):
                print(colored("\n📚 INFORMACIÓN DEL DESTINO:", "cyan", attrs=["bold"]))
                print(colored("─" * 60, "cyan"))
                print(current_state["destination_research"])
                print(colored("─" * 60 + "\n", "cyan"))

            if current_state.get("travel_itinerary"):
                print(colored("\n🗓️  TU ITINERARIO PERSONALIZADO:", "green", attrs=["bold"]))
                print(colored("═" * 60, "green"))
                print(current_state["travel_itinerary"])
                print(colored("═" * 60 + "\n", "green"))

            print(colored("✈️  ¡Buen viaje! 🌍", "cyan", attrs=["bold"]))
            break

        # Pedir input al usuario
        print(colored("👤 Tú: ", "white", attrs=["bold"]), end="")
        user_input = input().strip()

        # Comando para salir
        if user_input.lower() in ["salir", "exit", "quit"]:
            print(colored("\n👋 ¡Hasta luego!", "yellow"))
            break

        # Agregar mensaje del usuario
        current_state["messages"].append(HumanMessage(content=user_input))

        print(colored("\n" + "─"*60, "white"))

        # Ejecutar el grafo con el nuevo mensaje hasta el siguiente punto de interacción
        async for event in travel_graph.astream(current_state, config):
            for node_name, node_output in event.items():
                current_state = node_output

                # Si llegamos al interviewer de nuevo, parar aquí para pedir input
                if node_name == "interviewer" and not current_state.get("interview_complete"):
                    break

            # Salir del stream si llegamos al interviewer
            if node_name == "interviewer" and not current_state.get("interview_complete"):
                break


# ============================================================
# EJECUTAR EL SISTEMA
# ============================================================
print(colored("\n🚀 Iniciando sistema...\n", "green", attrs=["bold"]))

# Ejecutar en modo asíncrono
await run_travel_planner()


🚀 Iniciando sistema...


╔══════════════════════════════════════════════════════════╗
║                                                          ║
║    ✈️  PLANIFICADOR DE VIAJES MULTI-AGENTE ✈️           ║
║                                                          ║
║    Sistema: LangGraph                                    ║
║    Agentes: 4 especializados                             ║
║    Modo: Interactivo                                     ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝
    
💡 Tip: El sistema te hará preguntas para planificar tu viaje
💡 Responde naturalmente, como si hablaras con un agente de viajes

🔑 Sesión iniciada: travel_session_7a88af75

🔄 Iniciando conversación...


👤 [ENTREVISTADOR ACTIVADO]
🤖 Asistente: ¡Hola! 👋 Soy tu asistente personal de viajes. 

Estoy aquí para ayudarte a planificar el viaje perfecto. Para hacerlo, necesito conocer un poco sobre tus preferencias.

¿A qué des